# Install

In [1]:
pip install torch==1.13.1+cu116 torchvision==0.14.1+cu116 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu116

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 469.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.2/24.2 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1
    Uninstalling torchvision-0.15.1:
      Successfully uninstalled torchvision-0.15.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1
    Uninstalling torchaudio-2.0.1:
      Successfully uninstalled torchaudio-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 

In [2]:
pip install rtdl

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import sklearn.model_selection
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.dlpack import to_dlpack, from_dlpack
import rtdl
import urllib
import PIL
import requests
import datetime
import holidays
import os
import gc
import cudf as cf
import cupy as cp
from tqdm.auto import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
rng = cp.random.RandomState(seed=100)

# Data Preprocess

## Remove data outside of boundary

In [5]:
def select_within_boundary(df, boundary) -> bool:
    return (
        (df["pickup_longitude"] >= boundary["longitude_min"])
        & (df["pickup_longitude"] <= boundary["longitude_max"])
        & (df["pickup_latitude"] >= boundary["latitude_min"])
        & (df["pickup_latitude"] <= boundary["latitude_max"])
        & (df["dropoff_longitude"] >= boundary["longitude_min"])
        & (df["dropoff_longitude"] <= boundary["longitude_max"])
        & (df["dropoff_latitude"] >= boundary["latitude_min"])
        & (df["dropoff_latitude"] <= boundary["latitude_max"])
    )


def select_in_boundary(df: cf.DataFrame) -> cf.DataFrame:
    boundary = {
        "longitude_min": -74.5,
        "longitude_max": -72.8,
        "latitude_min": 40.5,
        "latitude_max": 41.8,
    }

    return df[select_within_boundary(df, boundary)]

## Drop data on water

In [6]:
mask_url = urllib.request.urlopen("https://imgur.com/XGHkdoK.png")
mask = np.array(PIL.Image.open(mask_url))[:, :, 0] > 0.92

mask = np.c_[mask, np.full([mask.shape[0], 1], False)]
mask = np.r_[mask, np.full([1, mask.shape[1]], False)]
mask = cp.asarray(mask)


def drop_on_water(df: cf.DataFrame) -> cf.DataFrame:
    def lonlat_to_xy(longitude, latitude, x_range, y_range, boundary):
        longitude_range = boundary["longitude_max"] - boundary["longitude_min"]
        latitude_range = boundary["latitude_max"] - boundary["latitude_min"]

        x = x_range * (longitude - boundary["longitude_min"]) / longitude_range
        y = (
            y_range
            - y_range * (latitude - boundary["latitude_min"]) / latitude_range
        )

        return (x.astype("uint8"), y.astype("uint8"))

    boundary = {
        "longitude_min": -74.5,
        "longitude_max": -72.8,
        "latitude_min": 40.5,
        "latitude_max": 41.8,
    }

    pickup_x, pickup_y = lonlat_to_xy(
        df.loc[:, "pickup_longitude"],
        df.loc[:, "pickup_latitude"],
        mask.shape[1] - 1,
        mask.shape[0] - 1,
        boundary,
    )

    dropoff_x, dropoff_y = lonlat_to_xy(
        df.loc[:, "dropoff_longitude"],
        df.loc[:, "dropoff_latitude"],
        mask.shape[1] - 1,
        mask.shape[0] - 1,
        boundary,
    )

    on_land = mask[pickup_y, pickup_x] & mask[dropoff_y, dropoff_x]
    return df[on_land]

In [7]:
def drop_same_pick_drop(df: cf.DataFrame):
    filter = (df["pickup_longitude"] == df["dropoff_longitude"]) & (
        df["pickup_latitude"] == df["dropoff_latitude"]
    )

    return df[~filter]


## Feature engineering

In [8]:
def get_lat_lon(df: cf.DataFrame, unit="deg"):
    # Return lat, lon in radian
    lat1 = df["pickup_latitude"].copy().to_cupy()
    lon1 = df["pickup_longitude"].copy().to_cupy()
    lat2 = df["dropoff_latitude"].copy().to_cupy()
    lon2 = df["dropoff_longitude"].copy().to_cupy()

    if unit == "rad":
        lat1, lon1, lat2, lon2 = map(cp.radians, [lat1, lon1, lat2, lon2])

    # 1 degree of latitude = 69.172 miles, 1 degree of longitude = 50 miles
    if unit == "mile":
        lat1 *= 69.172
        lon1 *= 50
        lat2 *= 69.172
        lon2 *= 50

    return lat1, lon1, lat2, lon2


def cal_rotated_coordinate(lat1, lon1, lat2, lon2) -> cp.ndarray:
    lat1, lon1, lat2, lon2 = map(
        lambda x: torch.as_tensor(x.astype("float32")).to(device), [lat1, lon1, lat2, lon2]
    )
    p1 = torch.column_stack([lat1, lon1])
    p2 = torch.column_stack([lat2, lon2])
    
    theta = -np.radians(29).astype("float32")

    rot = torch.tensor(
        [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
    ).to(device)

    # Perform rotate row by row and split
    lat1, lon1 = torch.hsplit(torch.einsum("ij, mj -> mi", rot, p1), 2)
    lat2, lon2 = torch.hsplit(torch.einsum("ij, mj -> mi", rot, p2), 2)
    lat1, lon1, lat2, lon2 = map(
        lambda x: cp.asarray(x.squeeze(1)),
        [lat1, lon1, lat2, lon2],
    )
    
    return lat1, lon1, lat2, lon2


def get_rotated_coordinate(df: pd.DataFrame):
    lat1, lon1, lat2, lon2 = get_lat_lon(df, "deg")
    header = [
        "rotated_pickup_latitude",
        "rotated_pickup_longitude",
        "rotated_dropoff_latitude",
        "rotated_dropoff_longitude",
    ]
    
    coordinates = cal_rotated_coordinate(lat1, lon1, lat2, lon2)

    for head, coordinate in zip(header, coordinates):
        df.loc[:, head] = coordinate

    return df

### Add distance (in miles)

Using Manhattan distance and rotate 29 degree to fit the real street block

In [9]:
def get_euclidean(df: cf.DataFrame):
    lat1, lon1, lat2, lon2 = get_lat_lon(df, "mile")
    return cp.linalg.norm(
        cp.column_stack([lat1, lon1]) - cp.column_stack([lat2, lon2]), axis=1
    )


def cal_haversine_distance(lat1, lon1, lat2, lon2):
    dlat = lat1 - lat2
    dlon = lon1 - lon2

    tmp = (
        cp.sin(dlat / 2.0) ** 2
        + cp.cos(lat1) * cp.cos(lat2) * cp.sin(dlon / 2.0) ** 2
    )

    return 2 * cp.arcsin(cp.sqrt(tmp))


def get_haversine_distance(df: cf.DataFrame):
    # Return haversine distance in miles
    lat1, lon1, lat2, lon2 = get_lat_lon(df, "rad")
    return cal_haversine_distance(lat1, lon1, lat2, lon2) * 3959


def get_correct_manhattan(df: cf.DataFrame):
    lat1, lon1, lat2, lon2 = get_lat_lon(df, "mile")
    lat1, lon1, lat2, lon2 = cal_rotated_coordinate(lat1, lon1, lat2, lon2)

    dlat = abs(lat1 - lat2)
    dlon = abs(lon1 - lon2)

    return (dlat + dlon).ravel()


### Add temperature and precipitation 

In [10]:
def get_historical_temp_precipitation():
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude=40.71&longitude=-74.01&start_date=2009-01-01&end_date=2015-12-31&hourly=apparent_temperature,precipitation"
    response = requests.get(url)
    data = response.json()

    df_tmp = cf.DataFrame(data["hourly"])
    df_tmp["time"] = cf.to_datetime(df_tmp["time"])

    return df_tmp.set_index("time").to_dict()


def convert_time(x: pd.Series) -> pd.Series:
    year = x.dt.year.rename("year")
    month = x.dt.month.rename("month").astype("uint8")
    day = x.dt.day.rename("day").astype("uint8")
    hour = x.dt.hour.rename("hour").astype("uint8")
    
    df_date = cf.concat([year, month, day, hour], axis=1)

    return cf.to_datetime(df_date)


def add_temp_precipitation(df: pd.DataFrame):
    date_time = convert_time(df["pickup_datetime"])
    df.loc[:,"apparent_temperature"] = date_time.map(
        temp_dict["apparent_temperature"]).astype("float32")
    df.loc[:, "precipitation"] = date_time.map(
        temp_dict["precipitation"]).astype("float32")

    return df


temp_dict = get_historical_temp_precipitation()


## Split time and add holiday

In [11]:
# add time information
def add_time_and_holiday_info(df: cf.DataFrame) -> cf.DataFrame:
    # Add time information
    df.loc[:, "year"] = df.pickup_datetime.dt.year
    df.loc[:, "weekday"] = df.pickup_datetime.dt.weekday.astype("uint8")
    df.loc[:, "hour"] = df.pickup_datetime.dt.hour.astype("uint8")

    # Add holiday information
    us_holidays = holidays.US()
    df.loc[:, "is_holiday"] = convert_time(df["pickup_datetime"]).isin(us_holidays).astype(
        "uint8"
    )

    return df


### Add pick_up/ drop_off airport feature

In [12]:
def check_nearby_airports(df, range=1):
    # Check the pickup and dropoff is near the airport (range in miles)
    # New York city
    nyc = (-74.0063889, 40.7141667)

    # JFK airport coordinates, see https://www.travelmath.com/airport/JFK
    jfk = (-73.7822222222, 40.6441666667)

    # Newark Liberty International Airport, see https://www.travelmath.com/airport/EWR
    ewr = (-74.175, 40.69)

    # LaGuardia Airport, see https://www.travelmath.com/airport/LGA
    lgr = (-73.87, 40.77)

    airports = {"JFK": jfk, "EWR": ewr, "LGR": lgr}

    # Add airport_nearby column
    near_pickup = cp.zeros(len(df))
    near_dropoff = cp.zeros(len(df))

    for airport, loc in airports.items():
        idx_pickup = (
            cal_haversine_distance(
                df["pickup_latitude"], df["pickup_longitude"], loc[1], loc[0]
            )
            * 3959
            < range
        )
        idx_dropoff = (
            cal_haversine_distance(
                df["dropoff_latitude"], df["dropoff_longitude"], loc[1], loc[0]
            )
            * 3959
            < range
        )

        if airport == "JFK":
            near_pickup[idx_pickup] = 1
            near_dropoff[idx_dropoff] = 1
        elif airport == "EWR":
            near_pickup[idx_pickup] = 2
            near_dropoff[idx_dropoff] = 2
        elif airport == "LGR":
            near_pickup[idx_pickup] = 3
            near_dropoff[idx_dropoff] = 3

    df["airport_nearby_pickup"] = near_pickup.astype("uint8")
    df["airport_nearby_dropoff"] = near_dropoff.astype("uint8")

    return df

# All Preprocess

In [13]:
def date_format(df: cf.DataFrame) -> cf.DataFrame:
    df = df.copy()
    date_time = df["pickup_datetime"].copy()

    date_time = date_time.str.slice(0, 16)
    date_time = cf.to_datetime(date_time, utc=True, format="%Y-%m-%d %H:%M")

    df["pickup_datetime"] = date_time
    return df


def clean_data(df: cf.DataFrame) -> cf.DataFrame:
    df = df.copy()
    # Drop negative fare amount
    df = df[df.fare_amount > 0]
    # Drop nan value
    df = df.dropna()
    # Drop data out of boundary
    df = select_in_boundary(df)
    # Drop data on water
    df = drop_on_water(df)
    # Drop same pickup and dropoff data
    df = drop_same_pick_drop(df)
    df = df.reset_index(drop=True)
    return df


def engineering(df: cf.DataFrame):
    progress = tqdm(total = 5, desc = "Feature Engineering: ")
    df = df.copy()
    # Add rotate coordinate
    df = get_rotated_coordinate(df)
    progress.update(1)
    # Add distance
    df.loc[:, "euclidean"] = get_euclidean(df)
    df.loc[:, "haversine"] = get_haversine_distance(df)
    df.loc[:, "correct_manhattan"] = get_correct_manhattan(df)
    progress.update(1)
    # Add Temp and precipitation
    df = add_temp_precipitation(df)
    progress.update(1)
    # Split time and add holiday
    df = add_time_and_holiday_info(df)
    progress.update(1)
    # Add pickup/dropoff airport
    df = check_nearby_airports(df)
    progress.update(1)

    return df


# FT Transformer

### Numerical, category, target split

In [14]:
numerical_idx = []
category_idx = []

In [15]:
def num_cat_split(df: pd.DataFrame, type="train"):
    x_num = torch.from_numpy(df.loc[:, numerical_idx].to_numpy()).to(device)
    x_cat = torch.from_numpy(df.loc[:, category_idx].to_numpy()).to(device)
    if type == "train":
        y = torch.from_numpy(df.loc[:, "fare_amount"].to_numpy()).to(device)

    return (x_num, x_cat, y) if type == "train" else (x_num, x_cat)

### Category data encoding

In [16]:
def cat_encoding(x_cat: torch.Tensor):
    for i, val in enumerate(x_cat.T):
        x_cat.T[i] = torch.from_numpy(
            LabelEncoder().fit_transform(val.cpu().numpy())
        )

    return x_cat.int()

### Train, validate split

In [17]:
def train_valid_spilt(
    x_num: torch.Tensor, x_cat: torch.Tensor, y: torch.Tensor
):
    x_num = x_num.cpu().numpy()
    x_cat = x_cat.cpu().numpy()
    y = y.cpu().numpy()

    X_num = {}
    X_cat = {}
    Y = {}

    (
        X_num["train"],
        X_num["val"],
        X_cat["train"],
        X_cat["val"],
        Y["train"],
        Y["val"],
    ) = sklearn.model_selection.train_test_split(
        x_num, x_cat, y, train_size=0.7, shuffle=True
    )
    X_num = {k: torch.from_numpy(v).to(device) for k, v in X_num.items()}
    X_cat = {k: torch.from_numpy(v).to(device) for k, v in X_cat.items()}
    Y = {k: torch.from_numpy(v).to(device) for k, v in Y.items()}

    return X_num, X_cat, Y

### Target standardize

In [18]:
class Standardizer:
    def __init__(self) -> None:
        pass

    def fit(self, x_num):
        self.mean = x_num.mean(dim=0)
        self.std = x_num.std(dim=0)

        return self

    def transform(self, x_num):
        return (x_num - self.mean) / self.std

    def get_param(self):
        return self.mean.item(), self.std.item()

## Premodel process

In [19]:
def premodel_process(df_train: pd.DataFrame, df_test: pd.DataFrame):
    progress = tqdm(total = 5, desc = "Premodel process: ")
    df_train = df_train.copy()
    df_test = df_test.copy()

    # Numerical, category data split
    x_num_train, x_cat_train, y_train = num_cat_split(df_train)
    x_num_test, x_cat_test = num_cat_split(df_test, type="test")
    progress.update(1)

    # Category data encoding
    x_cat_train = cat_encoding(x_cat_train)
    x_cat_test = cat_encoding(x_cat_test)
    progress.update(1)

    # Train, validate split and turn to numpy array
    X_num, X_cat, Y = train_valid_spilt(x_num_train, x_cat_train, y_train)

    X_num["test"] = x_num_test
    X_cat["test"] = x_cat_test
    progress.update(1)

    # Numerical data standardize
    standardizer = Standardizer().fit(X_num["train"])
    X_num = {k: standardizer.transform(v) for k, v in X_num.items()}

    # Target standardize
    standardizer = Standardizer().fit(Y["train"])
    Y = {k: standardizer.transform(v) for k, v in Y.items()}
    y_mean, y_std = standardizer.get_param()
    progress.update(1)

    cat_cardinalities = rtdl.data.get_category_sizes(X_cat["train"].cpu().numpy())
    progress.update(1)

    return X_num, X_cat, Y, y_mean, y_std, cat_cardinalities

In [20]:
train_path = "/kaggle/input/taxi-prediction-compress-train/compress_train.feather"
test_path =  "/kaggle/input/taxi-prediction-compress-train/compress_test.feather"
key_path = "/kaggle/input/new-york-city-taxi-fare-prediction/test.csv"

df_key = cf.read_csv(key_path, usecols=["key"])
# data_train = cf.read_feather(train_path).sample(frac=0., random_state = rng).reset_index(drop = True)
data_train = cf.read_feather(train_path)[:1_000_000]
data_test = cf.read_feather(test_path)

/opt/conda/lib/python3.10/site-packages/cudf/io/feather.py:15: UserWarning: Using CPU via PyArrow to read feather dataset, this may be GPU accelerated in the future
  warnings.warn(


In [21]:
df_train = clean_data(data_train)
df_train = engineering(df_train)
df_test = engineering(data_test)
gc.collect()

Feature Engineering:   0%|          | 0/5 [00:00<?, ?it/s]

Feature Engineering:   0%|          | 0/5 [00:00<?, ?it/s]

222

In [22]:
numerical_idx = [
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
#     "passenger_count",
#     "rotated_pickup_latitude",
#     "rotated_pickup_longitude",
#     "rotated_dropoff_latitude",
#     "rotated_dropoff_longitude",
#     "euclidean",
#     "haversine",
#     "correct_manhattan",
#     "apparent_temperature",
#     "precipitation",
]

category_idx = [
#     "year",
#     "weekday",
#     "hour",
#     "is_holiday",
    "airport_nearby_pickup",
    "airport_nearby_dropoff",
]

In [23]:
X_num, X_cat, Y, y_mean, y_std, cat_cardinalities = premodel_process(
    df_train, df_test
)
gc.collect()

Premodel process:   0%|          | 0/5 [00:00<?, ?it/s]

250

In [24]:
gc.collect()


0

### Pack data in dataset

In [25]:
class TaxiDataset(Dataset):
    def __init__(self, x_num, x_cat, y) -> None:
        self.x_num = x_num
        self.x_cat = x_cat
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        return self.x_num[index], self.x_cat[index], self.y[index]


### Model and Evaluater

In [26]:
def get_model():
    model = rtdl.FTTransformer.make_default(
        n_num_features = X_num["train"].shape[1],
        cat_cardinalities=cat_cardinalities,
        n_blocks = 5,
        last_layer_query_idx=[-1],
        d_out=1
    )


    model.to(device)

    optimizer = model.make_default_optimizer()
    loss_fn = F.smooth_l1_loss
    
    return model, optimizer, loss_fn


class EarlyStopper:
    def __init__(self, patience=100, min_delta=10e-5) -> None:
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_val_score = np.inf

    def early_stop(self, val_score) -> bool:
        if val_score < self.min_val_score:
            self.min_val_score = val_score
            self.counter = 0

        elif val_score >= (self.min_val_score + self.min_delta):
            self.counter += 1
            if self.counter > self.patience:
                return True

        return False


In [27]:
@torch.no_grad()
def evaluate(dataloader: DataLoader):
    model.eval()

    target = dataloader.dataset.y
    pred = []
    for data in tqdm(dataloader, miniters = 100, desc="Evaluate"):
        x_num_batch = data[0]
        x_cat_batch = data[1]

        pred.append(model(x_num_batch, x_cat_batch).squeeze(1))

    pred = torch.cat(pred)

    return sklearn.metrics.mean_squared_error(pred.cpu().numpy(), target.cpu().numpy()) ** 0.5 + y_std

@torch.no_grad()
def prediction():
    model.eval()
    y_pred = model(X_num["test"], X_cat["test"])
    return (y_pred * y_std) + y_mean


In [28]:
def train(n_epoch = 1):
    for epoch in range(1, n_epoch + 1):
        with tqdm(train_dataloader, miniters = 100, desc=f"Epoch {epoch}") as t: 
            for data in t:
                model.train()
                optimizer.zero_grad()

                x_num_batch = data[0]
                x_cat_batch = data[1]
                y_batch = data[2]

                loss = loss_fn(model(x_num_batch, x_cat_batch).squeeze(1), y_batch)
                loss.backward()
                optimizer.step()
                t.set_postfix(loss = loss.item())

        gc.collect()
        val_score = evaluate(val_dataloader)
        print(f"Epoch {epoch:03d} | Validation score: {val_score:.4f}")
        if early_stopper.early_stop(val_score):
            print(f"Early stop!")
            break

## Set dataloader

In [29]:
batch_size = 32

val_dataset = TaxiDataset(X_num["val"], X_cat["val"], Y["val"])
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

train_dataset = TaxiDataset(X_num["train"], X_cat["train"], Y["train"])
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True
)

early_stopper = EarlyStopper(min_delta=10e-6)

In [30]:
model, optimizer, loss_fn = get_model()

# Pre train score
evaluate(val_dataloader)

Evaluate:   0%|          | 0/9079 [00:00<?, ?it/s]

10.681353147732638

In [31]:
train(3)

Epoch 1:   0%|          | 0/21183 [00:00<?, ?it/s]

Evaluate:   0%|          | 0/9079 [00:00<?, ?it/s]

Epoch 001 | Validation score: 10.0906


Epoch 2:   0%|          | 0/21183 [00:00<?, ?it/s]

Evaluate:   0%|          | 0/9079 [00:00<?, ?it/s]

Epoch 002 | Validation score: 10.0880


Epoch 3:   0%|          | 0/21183 [00:00<?, ?it/s]

Evaluate:   0%|          | 0/9079 [00:00<?, ?it/s]

Epoch 003 | Validation score: 10.0820


### Get Prediction

In [32]:
cf.concat(
    [df_key, cf.Series(prediction().ravel(), name="fare_amount")], axis=1
).to_csv("/kaggle/working/output.csv", index=False)


In [33]:
pd.read_csv("/kaggle/working/output.csv")

,key,fare_amount
0,2015-01-27 13:08:24.0000002,8.549681
1,2015-01-27 13:08:24.0000003,9.108500
2,2011-10-08 11:53:44.0000002,5.229274
3,2012-12-01 21:12:12.0000002,7.889237
4,2012-12-01 21:12:12.0000003,14.869787
...,...,...
9909,2015-05-10 12:37:51.0000002,8.040543
9910,2015-01-12 17:05:51.0000001,8.112590
9911,2015-04-19 20:44:15.0000001,51.096886
9912,2015-01-31 01:05:19.0000005,18.587269
